**IMPORT LIBRARIES**

In [2]:
from datetime import datetime, timedelta

import plotly.graph_objs as go
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import satSimulationMagField as sim

import parallel_test4 as simulation
import plotMagneticField as hplot

**Run Satellite Orbit Propagator**

In [3]:
# TLE of the satellite
line1 = "1 00000U 20000A   25015.00000000  .00000000  00000-0  00000-0 0  9991"
line2 = "2 00000  97.3400 313.1310 0000000  0.0000  0.0000 15.09556778000000"

# Simulation parameters
start_date = datetime(2025, 1, 15)
end_date = start_date + timedelta(days=100)
time_step = timedelta(hours=0.08)

# Run the simulation
sim.initialize_satellite(line1, line2)
df = sim.simulate_satellite(start_date, end_date, time_step)

# Save results to CSV
df.to_csv("satellite_propagation_results.csv", index=False)
print("Results saved to 'satellite_propagation_results.csv'")

In [ ]:
df = pd.read_csv('satellite_propagation_results.csv')
values = sim.calculate_max_min_values(df, 'ECI')

In [5]:
# Initialize coil parameters
X_coil = simulation.CoilParameters(1, 0.5, 36)
Y_coil = simulation.CoilParameters(1, 0.5, 36)
Z_coil = simulation.CoilParameters(1, 0.5, 36)
#X_coil = test_3.CoilParameters(0.98083217, 0.95992063, 0.00128399)

# Current coil simulation
I  = np.array([1,2,3])
Ax = np.eye(3)
Ay = np.array([[0, -1,  0], [1,  0,  0], [0,  0,  1]])
Az = np.array([[0,  0, -1], [0,  1,  0], [1,  0,  0]])

# X coil simulation
grid_length_size = 0.1
num_seg = 100

In [ ]:
# Definir los argumentos adicionales
Ax = np.eye(3)  # Identity matrix
target_bx = 45839.86895741893  # Desired target for Bx
target_by = 45839.86895741893  # Desired target for By
target_bz = 45839.86895741893  # Desired target for Bz

# Bounds for the variables
my_bounds = [(0.4, 1), (0.4, 1)]  # Bounds for length and distance

# Tolerancia de 3 cm
tolerance_cm = 0.03  # 3 cm en unidades de metros

# Restricción dinámica: length no puede estar en un rango prohibido
def constraint_length_not_in_range(variables, forbidden_value, tolerance=0.1):
    """
    Restricción que asegura que `length` no esté dentro de [forbidden_value - tolerance, forbidden_value + tolerance].
    """
    length = variables[0]  # Extraer length de las variables
    return min(length - (forbidden_value + tolerance), (forbidden_value - tolerance) - length)

# First optimization (X)
constraints_x = []  # No forbidden value initially
length_x, distance_x = simulation.optimization_fun(Ax, target_bx, my_bounds, constraints_x)

# Update forbidden value for Y optimization
forbidden_value = length_x
constraints_y = [
    {
        "type": "ineq",
        "fun": constraint_length_not_in_range,
        "args": (forbidden_value, tolerance_cm)  # Rango alrededor de forbidden_value
    }
]

# Second optimization (Y)
length_y, distance_y = simulation.optimization_fun(Ax, target_by, my_bounds, constraints_y)

# Update forbidden values for Z optimization
forbidden_value1 = length_x
forbidden_value2 = length_y
constraints_z = [
    {
        "type": "ineq",
        "fun": constraint_length_not_in_range,
        "args": (forbidden_value1, tolerance_cm)  # Rango alrededor de forbidden_value1
    },
    {
        "type": "ineq",
        "fun": constraint_length_not_in_range,
        "args": (forbidden_value2, tolerance_cm)  # Rango alrededor de forbidden_value2
    }
]

# Third optimization (Z)
length_z, distance_z = simulation.optimization_fun(Ax, target_bz, my_bounds, constraints_z)

In [ ]:
##Generar espiras
spire1_x_s, spire2_x_s = simulation.square_spires(Ax, X_coil.h, X_coil.a, num_seg)
spire1_y_s, spire2_y_s = simulation.square_spires(Ay, Y_coil.h, Y_coil.a, num_seg)
spire1_z_s, spire2_z_s = simulation.square_spires(Az, Z_coil.h, Z_coil.a, num_seg)

f0 = None
f0 = hplot.plot_spires(f0, spire1_x_s, spire2_x_s, color='black')
f0 = hplot.plot_spires(f0, spire1_y_s, spire2_y_s, color='red')
f0 = hplot.plot_spires(f0, spire1_z_s, spire2_z_s, color='blue')
# Display the final figure
f0.update_layout(
        title="Gráficos de Superficie 3D - Todos los Planos",
)
f0.show()

In [ ]:
from scipy.optimize import differential_evolution
import numpy as np

# Objective function
def objective(variables, A, target_bx, forbidden, grid_length_size=0.01, num_seg=100):
    """
    Genetic algorithm-compatible objective function.
    """
    length, distance = variables  # Extract length and distance
    turns = 30  # Number of turns
    I = 1  # Current in the coil

    # Check if length is within any forbidden range
    for forbidden_value in forbidden:
        if (forbidden_value - 0.02) < length < (forbidden_value + 0.02):
            print(f'Forbidden: {variables}')
            return 1e6  # Large penalty for infeasible solutions
    
    # Initialize coil parameters
    coil = simulation.CoilParameters(length, distance, turns)

    # Generate coil spires and run simulation
    spire1_s, spire2_s = simulation.square_spires(A, coil.h, coil.a, num_seg)
    X = np.arange(-coil.a, coil.a, grid_length_size)  # X-axis points
    Y = np.zeros_like(X)  # Y-axis remains zero (1D simulation)

    # Run the simulation
    x_coil_results_s = simulation.coil_simulation_1d_sequential(
        X, Y, coil, I, spire1_s, spire2_s, 1, 20
    )
    bx_line = x_coil_results_s[
        (x_coil_results_s['Y'] == 0) & (x_coil_results_s['Z'] == 0)
    ]

    # Maximize range of Bx
    bx = bx_line['Bx'].max()
    e = target_bx - bx * 1e9  # Target error calculation

    if e <= 0:
        tolerance = 0.005 * bx
        lower_bound = bx - tolerance
        upper_bound = bx + tolerance
        filtered_points = bx_line[
            (bx_line['Bx'] >= lower_bound) & (bx_line['Bx'] <= upper_bound)
        ]

        # Check for contiguous points
        x_values = filtered_points['X'].sort_values()
        is_contiguous = all(
            (x_values.iloc[i + 1] - x_values.iloc[i]) <= 2 * grid_length_size
            for i in range(len(x_values) - 1)
        )

        if not is_contiguous:
            return 5000  # Penalty

        # Calculate range
        a = abs(filtered_points['X'].max() - filtered_points['X'].min())
        return -a  # Negative to maximize
    else:
        return 5000 + e  # Penalty for exceeding target


def opt_param(A, target, forbidden, bounds):
    # Run genetic algorithm using differential evolution
    result = differential_evolution(
        lambda variables: objective(variables, A, target_bx=target, forbidden=forbidden),
        bounds=bounds,
        strategy="best1bin",
        maxiter=10,  # Maximum number of generations
        popsize=10,  # Population size
        tol=1e-3,
        mutation=(0.5, 1),
        recombination=0.7,
        seed=None,
        disp=True,
    )
    return result

# Definir los argumentos adicionales
Ax = np.eye(3)  # Identity matrix
target_bx = 45839.86895741893  # Desired target for Bx
target_by = 45839.86895741893  # Desired target for By
target_bz = 45839.86895741893  # Desired target for Bz

# Bounds for the variables
my_bounds = [(0.25, 0.5), (0.25, 5)]  # Bounds for length and distance

forbidden_x = [0]
resultX = opt_param(Ax, target_bx, forbidden_x, my_bounds)

# Extract results
best_length_x, best_distance_x = resultX.x
print(f"Best solution: length = {best_length_x}, distance = {best_distance_x}, fitness = {-resultX.fun}")

forbidden_y = [best_length_x]
resultY = opt_param(Ax, target_by, forbidden_y, my_bounds)

# Extract results
#best_length_y, best_distance_y = resultY.x
#print(f"Best solution: length = {best_length_y}, distance = {best_distance_y}, fitness = {-resultY.fun}")

#forbidden_z = [best_length_x, best_length_y]
#resultZ = opt_param(Ax, target_bz, forbidden_z, my_bounds)

# Extract results
#best_length_z, best_distance_z = resultZ.x
#print(f"Best solution: length = {best_length_z}, distance = {best_distance_z}, fitness = {-resultZ.fun}")




In [10]:
forbidden_y = [best_length_x]
resultY = opt_param(Ax, target_by, forbidden_y, my_bounds)







Simulation Progress:   3%|▎         | 40/1225 [00:01<00:45, 26.30it/s]








Simulation Progress:   3%|▎         | 60/1936 [00:01<00:59, 31.60it/s]






Simulation Progress:   4%|▎         | 40/1089 [00:01<00:30, 33.89it/s]






Simulation Progress:   5%|▌         | 40/729 [00:01<00:17, 38.34it/s]






Simulation Progress:   4%|▎         | 60/1681 [00:01<00:43, 36.95it/s]








Simulation Progress:   3%|▎         | 60/1849 [00:01<00:46, 38.40it/s]






Simulation Progress:   5%|▌         | 40/784 [00:01<00:27, 27.03it/s]






Simulation Progress:   4%|▍         | 40/961 [00:01<00:43, 20.94it/s]






Simulation Progress:   5%|▍         | 40/841 [00:01<00:33, 23.70it/s]






Simulation Progress:   3%|▎         | 40/1296 [00:02<01:04, 19.56it/s]






Simulation Progress:   3%|▎         | 40/1369 [00:02<01:09, 19.04it/s]


Forbidden: [0.46166106 0.58059115]
Forbidden: [0.468903   1.16292073]








Simulation Progress:   3%|▎         | 40/1521 [00:02<01:47, 13.74it/s]








Simulation Progress:   3%|▎         | 60/1936 [00:03<01:39, 18.93it/s]


Forbidden: [0.47989812 4.58481929]








Simulation Progress:   2%|▎         | 40/1600 [00:03<01:57, 13.29it/s]






Simulation Progress:   3%|▎         | 40/1156 [00:02<01:08, 16.20it/s]


...







Simulation Progress:   3%|▎         | 40/1156 [00:01<00:51, 21.68it/s]


differential_evolution step 8: f(x)= -0.0








Simulation Progress:   3%|▎         | 40/1225 [00:01<00:55, 21.38it/s]






Simulation Progress:   5%|▍         | 40/841 [00:01<00:32, 24.39it/s]






Simulation Progress:   2%|▎         | 40/1600 [00:01<01:14, 21.05it/s]






Simulation Progress:   3%|▎         | 40/1444 [00:01<01:06, 21.11it/s]






Simulation Progress:   3%|▎         | 40/1156 [00:01<00:54, 20.33it/s]






Simulation Progress:   3%|▎         | 40/1156 [00:02<00:59, 18.73it/s]






Simulation Progress:   3%|▎         | 40/1444 [00:02<01:29, 15.70it/s]






Simulation Progress:   3%|▎         | 40/1156 [00:01<00:50, 22.07it/s]






Simulation Progress:   5%|▍         | 40/841 [00:01<00:26, 30.17it/s]






Simulation Progress:   3%|▎         | 40/1369 [00:02<01:10, 18.95it/s]






Simulation Progress:   3%|▎         | 40/1444 [00:01<01:08, 20.53it/s]






Simulation Progress:   3%|▎         | 40/1156 [00:01<00:47, 23.47it/s]






Simulation Progress:   4%|▍         | 40/961 [00:01<00:40, 23.00it/s]




differential_evolution step 9: f(x)= -0.0
Polishing solution with 'L-BFGS-B'








Simulation Progress:   3%|▎         | 40/1225 [00:02<01:03, 18.62it/s]






Simulation Progress:   3%|▎         | 40/1225 [00:02<01:24, 14.00it/s]






Simulation Progress:   3%|▎         | 40/1225 [00:02<01:13, 16.10it/s]


In [ ]:
# Initialize coil parameters
X_coil = simulation.CoilParameters(best_length_x, best_distance_x, 30)
Y_coil = simulation.CoilParameters(best_length_y, best_distance_y, 30)
Z_coil = simulation.CoilParameters(best_length_z, best_distance_z, 30)
#X_coil = test_3.CoilParameters(0.98083217, 0.95992063, 0.00128399)

# Current coil simulation
I  = np.array([1,2,3])
Ax = np.eye(3)
Ay = np.array([[0, -1,  0], [1,  0,  0], [0,  0,  1]])
Az = np.array([[0,  0, -1], [0,  1,  0], [1,  0,  0]])

# X coil simulation
grid_length_size = 0.1
num_seg = 100

##Generar espiras coil.h, coil.a
spire1_x_s, spire2_x_s = simulation.square_spires(Ax, X_coil.h, X_coil.a, num_seg)
spire1_y_s, spire2_y_s = simulation.square_spires(Ay, Y_coil.h, Y_coil.a, num_seg)
spire1_z_s, spire2_z_s = simulation.square_spires(Az, Z_coil.h, Z_coil.a, num_seg)

f0 = None
f0 = hplot.plot_spires(f0, spire1_x_s, spire2_x_s, color='black')
f0 = hplot.plot_spires(f0, spire1_y_s, spire2_y_s, color='red')
f0 = hplot.plot_spires(f0, spire1_z_s, spire2_z_s, color='blue')
# Display the final figure
f0.update_layout(
        title="Gráficos de Superficie 3D - Todos los Planos",
)
f0.show()
 
print(f"Best solution X: length = {best_length_x}, distance = {best_distance_x}, fitness = {-resultX.fun}")

print(f"Best solution Y: length = {best_length_y}, distance = {best_distance_y}, fitness = {-resultY.fun}")

print(f"Best solution Z: length = {best_length_z}, distance = {best_distance_z}, fitness = {-resultZ.fun}")

**Orbit propagator plots**

In [ ]:
hplot.plot_orbit(df, 'ECI')

In [ ]:
hplot.plot_magField_time(df, 'NED')